<h1><center>Module 2: ArcGIS Introduction</center></h1>

ArcGIS is a popular GIS (Geographic Information System) client which allows for users to display and run analytics on geographic information. ArcGIS is particularly well suited for Python integration as it has native Python support and the ArcPy package for automating geoprocessing tasks. This module will walk through how to interact with ArcMap via the Python Window, a brief walkthrough of various geoprocessing abilities, learn where to access more in depth information, and begin working on accessing shapefiles and databases via ArcPY. 

### 1.1 ArcMap and the Python Window
On a PC with ArcMap installed open up a blank map. Once open, you should see a screen similar to the following:
<img src="figs/arcMapBasic.JPG" width="750">
On the right is the "Table of Contents" which displays the various Layers currently displayed on the map. The Toolbar has various important tools such as "Identify", "Select", "Editor Toolbar", "Catalog", "ArcToolbox", and "Python". If you do not have these options available, navigate to the "Customize Mode" and add them to your Toolbar. For now, select the "Python" button which should launch the Python Window as shown:
<img src="figs/pythonWindow.JPG" width="750">
This Python Window is a self contained Python Interpretor, similar to the one native to Python explored in Module 1. You can execute commands, assign variables, and perform geoprocessing tasks using the power of ArcPy. However, you must first import arcpy in the following fashion:
<pre><code> >>>import arcpy</code></pre>

ArcPy is a package similar to the NumPy package explored in Module 1; importing this package grants access to all of the ArcPy functions. The documentation for ArcPy is extensive, and made available through the "ArcGIS Desktop Help" from within ArcMap as well as online. Each page has an example of how the tool would be implemented within Python either through the Python Window, or as a standalone script. ArcPy is available in a variety of versions, most notably a Python 2.7.X based version which comes with ArcMap and a Python 3.X version which comes with ArcGIS Pro. This can lead to some setup difficulties in running Python code outside of the Python Window contained within ArcMap/Pro; particularly through Jupyter notebooks. If you are on a computer which has ArcMap installed alongside a Python2 based Jupyter notebook, then you should be able to import ArcPy as you would any other package. However, any and all code that is referenced in these Modules can just as easily be copied into a text editor like Sublime, compiled, and ran outside of the Jupyter Notebook environment.

### 1.2 Workspaces, Map Documents, and Locks
In ArcGIS one of the most important things to keep in mind is what format is the target data in, where is it stored, and where is the target path. For this reason, understanding workspaces is critical to being successful at scripting geoprocessing tools later on. **Workspaces** are defined as the path to where the target datasets exist, and are typically a path to a folder or a geodatabase. For the sake of these Modules, most workspaces will reference a directory which will contain multiple shapefiles and subdirectories as opposed to a **geodatabase** which is one file that contains all of the related data. Typically, geodatabases are a cleaner method of storing GIS data but for learning Python scripting it is more intuitive to work with shapefiles and directories as the results are immediate and tangible. 

When you first load ArcMap and create a blank map, it is stored in a **map document** which has the file extension *.mxd*. MXD's are your first point of entry for manipulating GIS data, and should be established at the beginning of your script to help establish your working environment. This looks like the following:
<pre><code> >>> mxd = arcpy.mapping.MapDocument("CURRENT") </code></pre>

In the above example, when run through the Python Window will establish the currently opened MXD as the working Map Document. The *"CURRENT"* tag can only be used when the target MXD is open, otherwise the Map Document must be defined via the absolute path:
<pre><code> >>> mxd = arcpy.mapping.MapDocument("C:/PythonScriptingForGis/Map/Test_map.mxd") </code></pre>

Once a Map Document has been referenced, ArcMap locks the MXD which prevents any other programs from modifying the map as the script makes its own alterations. For this reason, it is recommended that the reference be deleted following the saving of any edits via the following:
<pre><code> >>> mxd.save() 
 >>> del mxd</code></pre>
 
Once a script completes the MXD reference is automatically removed, however keeping with this practice will become more important once Cursors are introduced. With the MXD loaded, multiple map properties can be accessed and edited and are explained in detail at the MapDocument help doc located [here.](https://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy-mapping/mapdocument-class.htm)

Establishing workspaces are similar to establishing the target map document, and allow access to all of the files housed in that space without requiring absolute paths. This is accomplished as follows:
<pre><code> >>> arcpy.env.workspace = "C:/PythonScriptingForGis/Map" </code></pre>


### 1.3 Working with Data
With the data located, it is equally important to understand the difference between storage methods. Most data is read in through either a geodatabase, or on a shapefile basis. **Shapefiles** are a type of vector data storage which contains all of the attribute data for the features present in the dataset. Shapefile information is typically viewed through **Layers**, which will appear in the Table of Contents (TOC) on the right of the ArcMap window. Layers are how ArcMap displays the data contained in their associated shapefiles, and typically contain visualization information and **not** attribute data. One might think of layers as a series of visual preferences which can be applied to a given dataset without having to create multiple instances of that data. For this reason, you can have multiple different layers which may display different aspects of the same information. 

If you have not done so already, download the CO_Wells.zip folder from Github repository and extract the folder. Once extracted, open the CO_Wells.mxd with Arcmap where you should be presented with a series of errors:
<img src="figs/missingData.JPG" width="1000">

This is an example of the importance of geodatabases - missplaced files. As the map structure we are employing is based purely on individual shapefiles, the mxd needs to be instructed as to the specific locations of each shapefile. This can be accomplished through the Python window utilizing [MapDocument.replaceWorkspaces()](http://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy-mapping/mapdocument-class.htm):
<pre><code> >>> import arcpy
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> workspacePath = mxd.filePath[:-12]
 >>> mxd.replaceWorkspaces("", "",  workspacePath, "SHAPEFILE_WORKSPACE")
 >>> mxd.save()
 >>> del mxd</code></pre>

Here we designate the current open map as the target map document, remove the absolute reference to the map document by removing the last 12 characters from the filePath name, and then replacing the map documents workspace for all shapefiles to be the default folder. Once run, you should see the following data displayed:
<img src="figs/dataFound.JPG" width="1000">

Go ahead and hide all layers except for the "Colorado_County_Boundaries" layer to expedite ArcMap's loading times. With all shapefiles located, their attribute tables can be accessed to view all the data per shapefile. This is accomplished by right-clicking any of the layers in the TOC and selecting "Open Attribute table". You should be able to see the following for the "Wells" layer:
<img src="figs/attributeTable.JPG" width="1000">

As you can see, all the information per well is represented as different columns, or **attributes**. This shapefile has each Well identified as **points** with corresponding object ids called **FID**. It is these values that hold all of the power of GIS, and through filtering/querying these values we can associate geographic areas with information of interest. For example, we can isolate particular wells of interest from the rest of the Wells shapefile through SQL queries and temporary layer creation. Lets say we want to examine all of the Wells where Chevron is the Operator:

<pre><code> 
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> arcpy.MakeFeatureLayer_management("Wells", "ChevronWells", "Operator = 'CHEVRON USA INC'")
</code></pre>

There are a few notable syntax choices in the above line. The [MakeFeatureLayer_management](http://desktop.arcgis.com/en/arcmap/10.3/tools/data-management-toolbox/make-feature-layer.htm) operator requires an input file, output file, and accepts a {where} clause (which in this case is an SQL query). Because we are using the Python Window within ArcMap the default workspace we reset previously is employed, allowing us to use relative pathes for "Wells" and "ChevronWells" rather than absolutes. When writing a tool outside of the ArcMap Python Window it is crucial to establish your workspace and identify your map document as done previously. The {where} clause above (third parameter) is a short SQL query, which is basically just a filtering operation being applied. An important syntax note here is that SQL queries when used in ArcPy are read in as Strings as a whole, and therefore any strings within that query need to be distinquished. This is accomplished through using varying patterns of quotations (above, the whole query is encapsulated in double quotes ("") and the Operator name to be evaluated is a separate string ('')). 

Remember that layers created in this way are only temporary unless saved using: <pre><code> mxd.save() </code></pre>
Saving the temporary layers is not required for additional processing if they are purely used as intermediary steps which saves computation time and disk space. Also remember that in this case we are dealing with **layers**, and not **shapefiles**. This means the above operation only represents a different view on the Wells.shp shapefile and contains no actual attribute data itself. In order to export data to a separate shapefile, a [FeatureClassToShapefile_conversion](http://desktop.arcgis.com/en/arcmap/10.3/tools/conversion-toolbox/feature-class-to-shapefile.htm) operation can be employed:
<pre><code> 
 >>> mxd = arcpy.mapping.MapDocument("CURRENT")
 >>> arcpy.env.workspace = mxd.filePath[:-12]
 >>> arcpy.FeatureClassToShapefile_conversion("ChevronWells", arcpy.env.workspace)
</code></pre>
This operation can be useful for slimming down massive datasets, particularly when the intention is to send the filtered data elsewhere. 

### 1.4 Module 2 Project
In the Mineral Rights industry it is in the best interest of potential buyers to keep up to date with the permit status of horizontal wellbores throughout their life cycles, particularly if the wells have been drilled yet. Typically the rights for land parcels will yield higher profits when purchased during the planning phase of the well which adds certain value to identifying and tracking those permits. 
**The Task:** Using the "Directional_Lines" shapefile and the ArcMap Python Window, create a shapefile which contains all of the Planned directional wellbores in the state of Colorado using the above methods. 

In [1]:
import arcpy
import os
mxd = arcpy.mapping.MapDocument(os.getcwd() + "\\CO_Wells\\CO_Wells.mxd")

In [3]:
default_dir = mxd.filePath
default_dir = default_dir.rsplit('\\', 1)[0]
arcpy.env.workspace = default_dir
arcpy.MakeFeatureLayer_management(default_dir + "\\" + "Directional_Lines.shp", "Planned_Directional_Lines", "Dir_Status = 'Planned'")
arcpy.FeatureClassToShapefile_conversion("Planned_Directional_Lines", default_dir)
del mxd